In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import os
import glob
from datetime import timedelta
from causalimpact import CausalImpact

In [9]:
data_path = "../../synthetic_data/data/dose_counterfactuals"
dataset_files = glob.glob(os.path.join(data_path, "insulin_factor_*.csv"))

datasets = {}
for file in dataset_files:
    factor = os.path.basename(file).replace("insulin_factor_", "").replace("_", ".").replace(".csv", "")
    
    # Load data
    df = pd.read_csv(file, index_col=0)
    
    # Convert index to datetime but store original datetime as a column
    df.index = pd.to_datetime(df.index)
    df['datetime'] = df.index.copy()
    
    # Convert datetime index to numerical timestamp format (seconds since epoch)
    # This avoids the datetime dtype compatibility issue
    df.index = df.index.astype(np.int64) // 10**9
    
    datasets[factor] = df
    
print(f"Loaded {len(datasets)} counterfactual datasets with insulin factors:")
for factor in sorted(datasets.keys()):
    print(f"  - {factor}: {len(datasets[factor])} data points")
    
# Helper function to convert time values within your analysis code
def to_timestamp(dt):
    """Convert datetime to timestamp integer for indexing"""
    if isinstance(dt, pd.Timestamp):
        return dt.value // 10**9
    return pd.Timestamp(dt).value // 10**9

Loaded 5 counterfactual datasets with insulin factors:
  - 0.8: 2016 data points
  - 0.9: 2016 data points
  - 1.0: 2016 data points
  - 1.1: 2016 data points
  - 1.2: 2016 data points


In [10]:
# Use the baseline dataset (1.0 factor) to identify insulin events
baseline_df = datasets.get("1.0")
if baseline_df is None:
    print("Error: Baseline dataset (factor 1.0) not found in datasets")
    print(f"Available factors: {list(datasets.keys())}")
    all_events = []
else:
    all_events = baseline_df.index[baseline_df['insulin'] > 0]

# Filter events to ones that are at least 1 hour apart
filtered_events = []
last_event = None

for event in all_events:
    filtered_events.append(event)
    last_event = event

print(f"Found {len(filtered_events)} filtered insulin events")
# Limit to first 5 events for analysis speed
filtered_events = filtered_events[:5]
print(f"Using first {len(filtered_events)} events for analysis")

Found 21 filtered insulin events
Using first 5 events for analysis


In [11]:


# Create a directory for saving visualizations if it doesn't exist
vis_dir = "../../output/dose_counterfactual_analysis"
os.makedirs(vis_dir, exist_ok=True)

# We'll store results for each insulin factor
factor_results = {}

for factor, data in datasets.items():
    print(f"\n{'='*50}")
    print(f"Analyzing insulin factor: {factor}")
    
    event_results = []
    
    for event_idx, event in enumerate(filtered_events):
        print(f"\nProcessing event {event_idx+1}/{len(filtered_events)} at {event}")
        
        # Define the window around the event
        window_start = event - pd.Timedelta("45min")
        window_end = event + pd.Timedelta("30min")
        
        try:
            # Extract window data
            window_data = data.loc[window_start:window_end].copy()
            
            # Check if window data is empty
            if window_data.empty:
                print(f"Skipping event - no data in specified window")
                continue
            
            window_data = window_data.ffill()
            
            print(f"Window data shape: {window_data.shape}")
            
            # Define pre/post periods
            pre_indices = window_data.index[window_data.index <= event]
            post_indices = window_data.index[window_data.index > event]
            
            if len(pre_indices) == 0 or len(post_indices) == 0:
                print(f"Skipping event - cannot establish pre/post boundaries (pre: {len(pre_indices)}, post: {len(post_indices)})")
                continue
            
            closest_pre = pre_indices.max()
            closest_post = post_indices.min()
            
            if pd.isna(closest_pre) or pd.isna(closest_post):
                print(f"Skipping event - cannot establish pre/post boundaries")
                continue
                
            pre_period = [window_data.index.min(), closest_pre]
            post_period = [closest_post, window_data.index.max()]
            
            # Clean data
            ci_data = window_data.copy()
            ci_data = ci_data.replace([np.inf, -np.inf], np.nan)
            ci_data = ci_data.dropna()
            ci_data = ci_data.loc[:, ci_data.nunique() > 1]

            
            print(f"Pre-period: {pre_period[0]} to {pre_period[1]}")
            print(f"Post-period: {post_period[0]} to {post_period[1]}")
            
            # Create diagnostic plot
            plt.figure(figsize=(12, 6))
            plt.plot(ci_data.index, ci_data['glucose'], 'b-', label='Glucose')
            plt.axvline(x=event, color='r', linestyle='--', label='Insulin')
            plt.title(f'Glucose Data for Insulin Factor {factor}, Event at {event}')
            plt.ylabel('Glucose (mg/dL)')
            plt.legend()
            plt.savefig(f"{vis_dir}/factor_{factor}_event_{event_idx}_data.png")
            plt.close()
            
            # Adjust pre-data if needed to avoid constant values
            pre_data = ci_data.loc[pre_period[0]:closest_pre].copy()
            for col in pre_data.columns:
                if pre_data[col].nunique() == 1:
                    constant_val = pre_data[col].iloc[0]
                    ci_data.loc[closest_pre, col] = constant_val + 1
                    print(f"Adjusted pre-data for column {col} to avoid constant value")
            
            # Check if we have enough predictors/covariates
            if ci_data.shape[1] <= 1:
                print("Error: CausalImpact requires at least one covariate/predictor variable")
                continue
            
            # Print data summary for debugging
            print("Dataset summary for CausalImpact:")
            print(f"Shape: {ci_data.shape}")
            print(f"Columns: {ci_data.columns.tolist()}")
            print(f"Number of unique values per column: {ci_data.nunique().to_dict()}")
            print(f"Pre-period: {pre_period}")
            print(f"Post-period: {post_period}")
            
            # Try creating and running CausalImpact
            try:
                # Create the CausalImpact object
                impact = CausalImpact(ci_data, pre_period, post_period)
                
                # Explicitly run the model (this is needed!)
                impact.run()
                
                if impact.inferences is None:
                    print("Error: CausalImpact model failed to generate inferences")
                    continue
                
                # Extract results
                post_inferences = impact.inferences.loc[impact.inferences.index >= post_period[0]]
                avg_effect = post_inferences['point_effects'].mean()
                cum_effect = post_inferences['post_cum_effects'].iloc[-1]
                
                # Save impact plot
                impact.plot()
                plt.gcf().savefig(f"{vis_dir}/factor_{factor}_event_{event_idx}_impact.png")
                plt.close()
                
                # Store results
                insulin_dose = None
                if event in window_data.index:
                    insulin_dose = window_data.loc[event, 'insulin']
                else:
                    insulin_dose = window_data['insulin'].sum()
                
                event_results.append({
                    'event_time': event,
                    'insulin_dose': insulin_dose,
                    'avg_effect': avg_effect,
                    'cum_effect': cum_effect
                })
                
                print(f"Success: Average effect: {avg_effect:.2f}, Cumulative effect: {cum_effect:.2f}")
                
            except Exception as e:
                print(f"Error in CausalImpact: {type(e).__name__}: {str(e)}")
                
                # Try a simpler approach - just compare means before and after
                try:
                    print("Attempting fallback to simple before-after comparison...")
                    pre_mean = ci_data.loc[pre_period[0]:pre_period[1], 'glucose'].mean()
                    post_mean = ci_data.loc[post_period[0]:post_period[1], 'glucose'].mean()
                    simple_effect = post_mean - pre_mean
                    
                    event_results.append({
                        'event_time': event,
                        'insulin_dose': window_data.loc[event, 'insulin'] if event in window_data.index else window_data['insulin'].sum(),
                        'avg_effect': simple_effect,
                        'cum_effect': simple_effect * len(ci_data.loc[post_period[0]:post_period[1]])
                    })
                except Exception as e2:
                    print(f"Simple before-after effect: {simple_effect:.2f}")
                    print(f"Even simple comparison failed: {type(e2).__name__}: {str(e2)}")
        
        except Exception as e:
            print(f"Error: {type(e).__name__}: {str(e)}")
    
    # Store results for this factor
    if event_results:
        factor_results[factor] = pd.DataFrame(event_results)
        print(f"\nSummary for insulin factor {factor}:")
        print(factor_results[factor])


Analyzing insulin factor: 0.8

Processing event 1/5 at 1704094200


TypeError: unsupported operand type(s) for -: 'int' and 'Timedelta'